# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115021e+02     1.762527e+00
 * time: 0.7011539936065674
     1     1.014694e+01     9.202050e-01
 * time: 2.0104520320892334
     2    -1.158459e+01     1.089217e+00
 * time: 2.6907410621643066
     3    -3.420833e+01     8.148994e-01
 * time: 3.6553211212158203
     4    -4.756921e+01     5.017690e-01
 * time: 4.6195690631866455
     5    -5.703280e+01     2.400482e-01
 * time: 5.591886043548584
     6    -5.991295e+01     1.337438e-01
 * time: 6.246628046035767
     7    -6.094118e+01     5.041202e-02
 * time: 6.886240005493164
     8    -6.129719e+01     7.925396e-02
 * time: 7.52836799621582
     9    -6.156929e+01     3.159036e-02
 * time: 8.185052156448364
    10    -6.175999e+01     2.464879e-02
 * time: 8.823089122772217
    11    -6.192013e+01     2.112315e-02
 * time: 9.462013959884644
    12    -6.199044e+01     1.967238e-02
 * time: 10.10650897026062
    13    -6.207350e+01     1.668313e-02
 * time: 10.765666007995605
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557692
    AtomicNonlocal      14.8522665
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666460990
